# Load dim_date từ Silver lên Gold

## Mục tiêu
- Load dữ liệu từ Silver.dim_date → Gold.dim_date
- Không cần biến đổi dữ liệu
- Chỉ cần copy và load vào Gold database

## Quy trình 3 bước
1. **Load from Silver** - Đọc dữ liệu từ Silver database
2. **Schema Definition** - Định nghĩa schema cho MySQL
3. **Load to Gold** - Load vào Gold database


## Bước 1: Import và Setup


In [1]:
# Import các thư viện cần thiết
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_SILVER = os.getenv("DB_SILVER")
DB_GOLD = os.getenv("DB_GOLD")

# Tạo kết nối tới Silver và Gold database
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")
gold_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_GOLD}")

print("Đã kết nối Silver và Gold database thành công")
print(f"Silver DB: {DB_SILVER}")
print(f"Gold DB: {DB_GOLD}")
print(f"Thời gian bắt đầu: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Đã kết nối Silver và Gold database thành công
Silver DB: winner_silver
Gold DB: winner_gold
Thời gian bắt đầu: 2025-10-17 23:18:29


## Bước 2: Load dữ liệu từ Silver


In [2]:
# Load dữ liệu dim_date từ Silver database
print("=== LOADING DIM_DATE FROM SILVER ===")

# Load toàn bộ bảng dim_date
date_df = pd.read_sql("SELECT * FROM dim_date", silver_engine)

# Hiển thị thông tin cơ bản về dataset
print(f"Shape: {date_df.shape}")
print(f"Columns: {list(date_df.columns)}")
print(f"Memory usage: {date_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Hiển thị sample data
print("\n=== SAMPLE DATA ===")
print(date_df.head(5))

# Hiển thị data types
print("\n=== DATA TYPES ===")
print(date_df.dtypes)

# Kiểm tra date range
print(f"\n=== DATE RANGE ===")
print(f"Min date: {date_df['full_date'].min()}")
print(f"Max date: {date_df['full_date'].max()}")
print(f"Total days: {len(date_df)}")

# Kiểm tra null values
print(f"\n=== NULL VALUES ===")
null_counts = date_df.isnull().sum()
print(null_counts[null_counts > 0])


=== LOADING DIM_DATE FROM SILVER ===
Shape: (1386, 26)
Columns: ['date_id', 'full_date', 'year', 'month', 'day', 'day_of_week', 'weekday_name', 'day_of_year', 'week_of_year', 'quarter', 'quarter_name', 'month_name', 'year_month', 'year_quarter', 'is_weekend', 'is_weekday', 'is_holiday', 'holiday_name', 'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end', 'is_year_start', 'is_year_end', 'days_from_start', 'days_to_end']
Memory usage: 0.70 MB

=== SAMPLE DATA ===
    date_id   full_date  year  month  day  day_of_week weekday_name  \
0  20211130  2021-11-30  2021     11   30            1       Thứ Ba   
1  20211201  2021-12-01  2021     12    1            2       Thứ Tư   
2  20211202  2021-12-02  2021     12    2            3      Thứ Năm   
3  20211203  2021-12-03  2021     12    3            4      Thứ Sáu   
4  20211204  2021-12-04  2021     12    4            5      Thứ Bảy   

   day_of_year  week_of_year  quarter  ... is_holiday holiday_name  \
0          334    

## Bước 3: Load vào Gold Database


In [3]:
# Load vào Gold Database
print("=== LOADING TO GOLD DATABASE ===")

from sqlalchemy.types import Integer, Date, String, Boolean

# Định nghĩa schema mapping cho MySQL
dtype_mapping = {
    'date_id': Integer,
    'full_date': Date,
    'year': Integer,
    'month': Integer, 
    'day': Integer,
    'day_of_week': Integer,
    'weekday_name': String(10),
    'day_of_year': Integer,
    'week_of_year': Integer,
    'quarter': Integer,
    'quarter_name': String(3),
    'month_name': String(10),
    'year_month': String(7),
    'year_quarter': String(7),
    'is_weekend': Boolean,
    'is_weekday': Boolean,
    'is_holiday': Boolean,
    'holiday_name': String(50),
    'is_month_start': Boolean,
    'is_month_end': Boolean,
    'is_quarter_start': Boolean,
    'is_quarter_end': Boolean,
    'is_year_start': Boolean,
    'is_year_end': Boolean,
    'days_from_start': Integer,
    'days_to_end': Integer
}

print(f"Schema mapping defined for {len(dtype_mapping)} columns")

# Load vào Gold database
table_name = "dim_date"

try:
    date_df.to_sql(
        table_name,
        con=gold_engine,
        if_exists="replace",  # Ghi đè dữ liệu cũ
        index=False,
        dtype=dtype_mapping
    )
    
    print(f"Đã load {date_df.shape[0]} records vào Gold: {table_name}")
    
    # Verify load
    verification_query = f"SELECT COUNT(*) as count FROM {table_name}"
    verification_result = pd.read_sql(verification_query, gold_engine)
    loaded_count = verification_result['count'].iloc[0]
    
    print(f"Verification: {loaded_count} records trong database")
    print(f"Schema: {len(dtype_mapping)} columns với explicit typing")
    
    # Sample data từ database
    sample_query = f"SELECT date_id, full_date, year, month, weekday_name, is_weekend, is_holiday FROM {table_name} LIMIT 5"
    sample_db = pd.read_sql(sample_query, gold_engine)
    print(f"\nSample data từ Gold database:")
    print(sample_db)
    
    # Kiểm tra date range trong Gold
    range_query = f"SELECT MIN(full_date) as min_date, MAX(full_date) as max_date, COUNT(*) as total_days FROM {table_name}"
    range_result = pd.read_sql(range_query, gold_engine)
    print(f"\nDate range trong Gold database:")
    print(f"Min date: {range_result['min_date'].iloc[0]}")
    print(f"Max date: {range_result['max_date'].iloc[0]}")
    print(f"Total days: {range_result['total_days'].iloc[0]}")
    
except Exception as e:
    print(f"Error loading to Gold: {str(e)}")
    raise

print(f"\nLoad dim_date completed successfully!")
print(f"Ready for EDA and analytics")


=== LOADING TO GOLD DATABASE ===
Schema mapping defined for 26 columns
Đã load 1386 records vào Gold: dim_date
Verification: 1386 records trong database
Schema: 26 columns với explicit typing

Sample data từ Gold database:
    date_id   full_date  year  month weekday_name  is_weekend  is_holiday
0  20211130  2021-11-30  2021     11       Thứ Ba           0           0
1  20211201  2021-12-01  2021     12       Thứ Tư           0           0
2  20211202  2021-12-02  2021     12      Thứ Năm           0           0
3  20211203  2021-12-03  2021     12      Thứ Sáu           0           0
4  20211204  2021-12-04  2021     12      Thứ Bảy           1           0

Date range trong Gold database:
Min date: 2021-11-30
Max date: 2025-09-15
Total days: 1386

Load dim_date completed successfully!
Ready for EDA and analytics


## Bước 4: Tạo Data Dictionary cho Gold


In [4]:
# Tạo Data Dictionary cho Gold
print("=== CREATING GOLD DATA DICTIONARY ===")

def get_business_meaning_vietnamese(column_name):
    """Get business meaning for each column in Vietnamese"""
    business_meanings = {
        # Định danh & Cơ bản
        "date_id": "Mã định danh ngày duy nhất (khóa chính)",
        "full_date": "Ngày đầy đủ (YYYY-MM-DD)",
        
        # Thông tin năm tháng ngày
        "year": "Năm (4 chữ số)",
        "month": "Tháng (1-12)",
        "day": "Ngày trong tháng (1-31)",
        "day_of_week": "Thứ trong tuần (1=Thứ 2, 7=Chủ nhật)",
        "weekday_name": "Tên thứ trong tuần (Thứ 2, Thứ 3, ..., Chủ nhật)",
        "day_of_year": "Ngày thứ mấy trong năm (1-366)",
        "week_of_year": "Tuần thứ mấy trong năm (1-53)",
        
        # Thông tin quý
        "quarter": "Quý (1-4)",
        "quarter_name": "Tên quý (Q1, Q2, Q3, Q4)",
        
        # Thông tin tháng
        "month_name": "Tên tháng (Tháng 1, Tháng 2, ..., Tháng 12)",
        "year_month": "Năm-Tháng (YYYY-MM)",
        "year_quarter": "Năm-Quý (YYYY-Q)",
        
        # Thông tin ngày nghỉ và đặc biệt
        "is_weekend": "Có phải cuối tuần (true/false)",
        "is_weekday": "Có phải ngày thường (true/false)",
        "is_holiday": "Có phải ngày lễ (true/false)",
        "holiday_name": "Tên ngày lễ (nếu có)",
        
        # Thông tin đầu/cuối kỳ
        "is_month_start": "Có phải đầu tháng (true/false)",
        "is_month_end": "Có phải cuối tháng (true/false)",
        "is_quarter_start": "Có phải đầu quý (true/false)",
        "is_quarter_end": "Có phải cuối quý (true/false)",
        "is_year_start": "Có phải đầu năm (true/false)",
        "is_year_end": "Có phải cuối năm (true/false)",
        
        # Thông tin khoảng cách
        "days_from_start": "Số ngày từ đầu năm",
        "days_to_end": "Số ngày đến cuối năm"
    }
    return business_meanings.get(column_name, "Chưa định nghĩa ý nghĩa business")

# Tạo Data Dictionary
dict_data = []
for col in date_df.columns:
    col_info = {
        "table_name": "dim_date",
        "column_name": col,
        "dtype": str(date_df[col].dtype),
        "sql_type": str(dtype_mapping.get(col, "Not defined")),
        "null_count": date_df[col].isnull().sum(),
        "null_pct": round(date_df[col].isnull().mean() * 100, 2),
        "unique_count": date_df[col].nunique(),
        "sample_values": str(date_df[col].dropna().unique()[:3].tolist()),
        "business_meaning": get_business_meaning_vietnamese(col),
        "extraction_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    dict_data.append(col_info)

data_dictionary = pd.DataFrame(dict_data)

# Hiển thị Data Dictionary
print(f"Generated Data Dictionary for {len(data_dictionary)} columns")
print("\n=== DATA DICTIONARY ===")
print(data_dictionary)

# Lưu vào file Excel
excel_path = "Technical_Document/Gold_dictionary.xlsx"

try:
    from openpyxl import load_workbook
    
    # Kiểm tra file Excel có tồn tại không
    try:
        # Load workbook hiện tại
        wb = load_workbook(excel_path)
        
        # Lấy sheet đầu tiên
        ws = wb.active
        
        # Kiểm tra xem có dữ liệu cũ không
        if ws.max_row > 1:
            print(f"Found existing data in {excel_path}, appending new data...")
        else:
            print(f"File {excel_path} exists but is empty, adding header and data...")
            
    except FileNotFoundError:
        print(f"File {excel_path} not found, creating new file...")
        wb = None
    except Exception as e:
        print(f"Error loading {excel_path}: {str(e)}, creating new file...")
        wb = None
    
    if wb is None:
        # Tạo file mới với header
        data_dictionary.to_excel(excel_path, index=False, sheet_name='Gold_Dictionary')
        print(f"✅ Created new file: {excel_path}")
    else:
        # Append vào file hiện tại
        from openpyxl.utils.dataframe import dataframe_to_rows
        
        # Tìm dòng cuối cùng có dữ liệu
        last_row = ws.max_row
        
        # Thêm dữ liệu mới từ dòng tiếp theo
        for r in dataframe_to_rows(data_dictionary, index=False, header=False):
            last_row += 1
            for c_idx, value in enumerate(r, 1):
                ws.cell(row=last_row, column=c_idx, value=value)
        
        # Lưu file
        wb.save(excel_path)
        print(f"✅ Appended {len(data_dictionary)} rows to: {excel_path}")
    
except Exception as e:
    print(f"❌ Error appending to Data Dictionary: {str(e)}")
    # Fallback: tạo file mới
    try:
        data_dictionary.to_excel(excel_path, index=False)
        print(f"✅ Created new file as fallback: {excel_path}")
    except Exception as e2:
        print(f"❌ Error creating fallback file: {str(e2)}")

# Summary Report
print(f"\n=== GOLD DICTIONARY SUMMARY ===")
print(f"Table: dim_date")
print(f"Total columns: {len(data_dictionary)}")
print(f"Data Dictionary: {excel_path}")
print(f"Created at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n=== SAMPLE DICTIONARY ENTRIES ===")
sample_dict = data_dictionary[['column_name', 'dtype', 'business_meaning']].head(5)
print(sample_dict)


=== CREATING GOLD DATA DICTIONARY ===
Generated Data Dictionary for 26 columns

=== DATA DICTIONARY ===
   table_name       column_name   dtype  \
0    dim_date           date_id   int64   
1    dim_date         full_date  object   
2    dim_date              year   int64   
3    dim_date             month   int64   
4    dim_date               day   int64   
5    dim_date       day_of_week   int64   
6    dim_date      weekday_name  object   
7    dim_date       day_of_year   int64   
8    dim_date      week_of_year   int64   
9    dim_date           quarter   int64   
10   dim_date      quarter_name  object   
11   dim_date        month_name  object   
12   dim_date        year_month  object   
13   dim_date      year_quarter  object   
14   dim_date        is_weekend   int64   
15   dim_date        is_weekday   int64   
16   dim_date        is_holiday   int64   
17   dim_date      holiday_name  object   
18   dim_date    is_month_start   int64   
19   dim_date      is_month_end   in